In [1]:
import tensorflow as tf

import pandas as pd
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
#from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Add, Multiply, Subtract
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, BatchNormalization
# regularizers
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from scipy import signal
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pylab as plt
import matplotlib.dates as mdates
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc
from keijzer import *
from tensorflow import keras
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dense, Input, Flatten, Add, concatenate, Dropout, Activation, Multiply, Embedding, Layer, Reshape
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D  
from keras.layers import Conv1D, AveragePooling1D, MaxPooling1D
#from keras.ops import convert_to_tensor, convert_to_numpy
#from keras.utils import plot_model
from keras import activations
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import math

class LossHistory(tensorflow.keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.rmses = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_rmses = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.rmses.append(logs.get('root_mean_squared_error'))
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_rmses.append(logs.get('val_root_mean_squared_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))



def root_squared_mean_error(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))+K.mean(K.abs(y_pred - y_true))

from keras.layers.advanced_activations import PReLU

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.experimental.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print(e)

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Import Data & Preprocessing

In [3]:
data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2020.csv'
data_source1 = pd.read_csv(data_path)
dat_source1 = pd.DataFrame(data_source1)
#dat_source1 = dat_source1.iloc[:-3,:]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2021.csv'
data_source2 = pd.read_csv(data_path)
dat_source2 = pd.DataFrame(data_source2)
dat_source2 = dat_source2.iloc[:8761,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2022.csv'
data_source3 = pd.read_csv(data_path)
dat_source3 = pd.DataFrame(data_source3)
dat_source3 = dat_source3.iloc[:8760,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2023.csv'
data_source4 = pd.read_csv(data_path)
dat_source4 = pd.DataFrame(data_source4)
dat_source4 = dat_source4.iloc[:8760,:9]

dat_arr1 = np.array(dat_source1)
dat_arr2 = np.array(dat_source2)
dat_arr3 = np.array(dat_source3)
dat_arr4 = np.array(dat_source4)

dat_arr = np.concatenate([dat_arr1, dat_arr2, dat_arr3, dat_arr4],axis=0)
dat_arr.shape # 8784, 8761, 8760, 8760

dat_source = pd.DataFrame(dat_arr, columns=['Time-Date stamp','Date','ERCOT Load','Total Wind Output','Total Wind Installed','Wind Output, % of Load','Wind Output, % of Installed','1-hr MW change','1-hr % change'])
dat_source

,Time-Date stamp,Date,ERCOT Load,Total Wind Output,Total Wind Installed,"Wind Output, % of Load","Wind Output, % of Installed",1-hr MW change,1-hr % change
0,01-01-2020 00:00:00,Jan-01,36823,12335,27040,33.5,45.617,NaN,NaN
1,01-01-2020 01:00:00,Jan-01,36511,13300,27040,36.4,49.187,965.4,7.826
2,01-01-2020 02:00:00,Jan-01,36022,13586,27040,37.7,50.245,286.2,2.152
3,01-01-2020 03:00:00,Jan-01,35790,14107,27040,39.4,52.171,520.9,3.834
4,01-01-2020 04:00:00,Jan-01,35642,14887,27040,41.8,55.055,779.6,5.527
...,...,...,...,...,...,...,...,...,...
35060,12-31-2023 20:00:00,Dec-31,44887,17161,38631,38.23,44.42,2858,19.99
35061,12-31-2023 21:00:00,Dec-31,44013,18301,38631,41.58,47.37,1139,6.64
35062,12-31-2023 22:00:00,Dec-31,43280,18607,38631,42.99,48.17,307,1.67
35063,12-31-2023 23:00:00,Dec-31,42319,17866,38631,42.22,46.25,-741,-3.98


### Dividing by Installed Capacity & Preprocessing

In [4]:
dat = pd.DataFrame()
dat['MW % change'] = np.array(dat_source.iloc[1:,3], dtype=float)-np.array(dat_source.iloc[0:-1,3], dtype=float)
dat['MW % change'] = np.divide(np.array(dat['MW % change']),np.array(dat_source.iloc[1:,4]))*100
dat['Total Wind Output'] = np.divide(np.array(dat_source.iloc[1:,3]), np.array(dat_source.iloc[1:,4]))

dat

,MW % change,Total Wind Output
0,3.56879,0.491864
1,1.05769,0.502441
2,1.92678,0.521709
3,2.88462,0.550555
4,-0.580621,0.544749
...,...,...
35059,7.3982,0.444229
35060,2.951,0.473739
35061,0.79211,0.48166
35062,-1.91815,0.462478


In [5]:
ramp_ratio = 20
load_ratio = 10
year_arr = np.concatenate([np.ones(8784)*2020, np.ones(8760)*2021, np.ones(8760)*2022, np.ones(8760)*2023])
year_df = pd.DataFrame(year_arr, columns=['Year'], dtype=int)
wind_df = pd.concat([year_df, dat], axis=1)
wind_df

,Year,MW % change,Total Wind Output
0,2020,3.56879,0.491864
1,2020,1.05769,0.502441
2,2020,1.92678,0.521709
3,2020,2.88462,0.550555
4,2020,-0.580621,0.544749
...,...,...,...
35059,2023,7.3982,0.444229
35060,2023,2.951,0.473739
35061,2023,0.79211,0.48166
35062,2023,-1.91815,0.462478


In [6]:
wind_df = wind_df.reset_index(drop=True)
std_scaler = sklearn.preprocessing.MinMaxScaler()
std_scaler_load = sklearn.preprocessing.MinMaxScaler()
std_scaler_ramp = sklearn.preprocessing.MinMaxScaler()

wind_df[['Total Wind Output']] = std_scaler.fit_transform(wind_df[['Total Wind Output']])
wind_df[['MW % change']] = std_scaler_ramp.fit_transform(wind_df[['MW % change']])
#wind_df[['Wind Change, % of Load', 'Wind Change']] = std_scaler_ramp.fit_transform(wind_df[['Wind Change, % of Load', 'Wind Change']])
wind_df = wind_df.iloc[:-1,:]
wind_df

,Year,MW % change,Total Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


In [7]:
ramp_df = pd.DataFrame(wind_df)
ramp_df.columns=['Year', 'Wind Change', 'Wind Output']
ramp_df

,Year,Wind Change,Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


## Wind Ramp Rate Forecasting

### SVD-based Feature Generation

In [8]:
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc

def create_dataset(dataset, timesteps, output_timesteps, leadtime):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps -output_timesteps - leadtime - 1):
        a = dataset[i:(i + timesteps), :]
        dataX.append(a)
        dataY.append(dataset[(i + timesteps + leadtime):(i+timesteps+output_timesteps+leadtime), :])
    return np.array(dataX), np.array(dataY)

In [9]:
norm_df = np.array(wind_df['Wind Change']).reshape(-1,1)
timesteps = 24*6
output_timesteps = 12
num_features = 1
leadtime = 4
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps

In [10]:
dec_num = timesteps
high_ind = 35

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

def svdadd(X, start, end, b_size):
    dec_x = X[start:end, :, :]
    U, S, Vh = svd(dec_x.reshape(b_size, -1))

    high_sig = np.matmul(np.matmul(U[:,:high_ind], np.diag(S[0:high_ind])), Vh[:high_ind,:])
    low_sig = X[:dec_num, :, :].reshape(b_size,-1)-high_sig

    rec_x = np.zeros((b_size, b_size, high_ind))
    #print(sum(S[0:high_ind])/sum(S))
    for i in range(high_ind):
        rec_x[:,:,i] = np.matmul((U[:,i]*S[i]).reshape(-1,1), Vh[i,:].reshape(1,-1))    
    return rec_x, low_sig

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

### Training/Test Set Division & Shuffled Learning

In [11]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)

idx = np.arange(trvaX.shape[0])
np.random.shuffle(idx)
strvaX = trvaX[idx]
strvaY = trvaY[idx]
strvaY = strvaY.reshape(-1, output_timesteps)

trvaY = trvaY.reshape(-1, output_timesteps)
teY = teY.reshape(-1, output_timesteps)

strX, svaX, strY, svaY = train_test_split(strvaX, strvaY, test_size=0.3, shuffle=False)
trX, vaX, trY, vaY = train_test_split(trvaX, trvaY, test_size=0.3, shuffle=False)

trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31411, 144, 37), (31411, 12), (3491, 144, 37), (3491, 12))

## Wind Ramp Rate Forecasting Basic Model

In [12]:
def MAEMD(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

In [13]:
from keras.layers.advanced_activations import PReLU

In [14]:
from keras import regularizers

num_features = trvaX.shape[2]
output_timesteps = trvaY.shape[1]

with tf.device('/gpu:0'):
    alpha=1.0
    gamma=1
    beta = 1
    hfilters = 168
    hkernel_size1 = 1
    hkernel_size2 = 2
    visible1 = Input(shape=(timesteps, num_features))
   
    ## Attention Block
    pera = Permute((2,1))(visible1)
    den1a = Dense(timesteps, activation='tanh')(pera)
    den1b = Dense(timesteps, activation='sigmoid')(pera)
    den1 = Multiply()([den1a, den1b])
    per2 = Permute((2,1), name='attention_vec')(den1)
    mul1 = Multiply()([visible1, per2])   
    
    ## Permuted Attention Block
    per1 = Permute((2,1))(visible1)
    den2a = Dense(num_features, activation='tanh')(visible1)
    den2b = Dense(num_features, activation='sigmoid')(visible1)
    den2 = Multiply()([den2a, den2b])
    perb2 = Permute((2,1), name='attention_vec2')(den2)
    
    mul2 = Multiply()([per1, perb2])     
    
    ## Parallel DCCNN Blocks 1-1 ~ 3-1
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=1)(mul1)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    rres01a = Add()([mul1, d1])   # (100, 25) (100, 25)
    rres01b = Subtract()([mul1, d1])
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(rres01a)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    
    d1 = Multiply()([d1a, d1b])
    rres02a = Add()([rres01a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(rres01b) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    rres02b = Subtract()([rres01b, d2])   # (100, 25) (100, 25) 
    rres02 = Concatenate()([rres02a, rres02b, rres01a, rres01b])
    #rres02 = Dropout(0.2)(rres02)
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(rres02)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    rres03a = Add()([rres02a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(rres02) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    rres03b = Subtract()([rres02b, d2])
    rres10 = Concatenate()([rres03a, rres03b, rres02])

    ## Right-half DCCNN Blocks 1-2 ~ 3-2
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=1)(mul2)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    res01a = Add()([mul2, d1])   # (100, 25) (100, 25)
    res01b = Subtract()([mul2, d1])
    
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01a)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    
    d1 = Multiply()([d1a, d1b])
    res02a = Add()([res01a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01b) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    res02b = Subtract()([res01b, d2])   # (100, 25) (100, 25) 
    res02 = Concatenate()([res02a, res02b, res01a, res01b])
    #res02 = Dropout(0.2)(res02)
    
    d1 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02)
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    d1 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1)    
    d1a = Activation(activations.tanh)(d1)
    d1b = Activation(activations.sigmoid)(d1)
    d1 = Multiply()([d1a, d1b])
    
    res03a = Add()([res02a, d1])   # (100, 25) (100, 25)
    
    d2 = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    d2 = Conv1D(filters=timesteps, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d2) 
    d2a = Activation(activations.tanh)(d2)
    d2b = Activation(activations.sigmoid)(d2)
    d2 = Multiply()([d2a, d2b])
    
    res03b = Subtract()([res02b, d2])
    res10 = Concatenate()([res03a, res03b, res02])

    ## Output Blocks
    out1 = Conv1D(24*7, 1, padding='same', activation=PReLU())(res10)   # 256, 11X10=110
    out1 = Dropout(0.2)(out1)  
    
    out2 = Conv1D(24*7, 1, padding='same', activation=PReLU())(rres10)   # 256, 11X10=110
    out2 = Dropout(0.2)(out2)
    
    out = Concatenate(axis=1)([out1, out2])
    
    ## Output Block
    out = Conv1D(24*7, 1, padding='same', activation=PReLU())(out) # 512,  110X5=550
    out = Dropout(0.2)(out)

    #, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
    out = GlobalAveragePooling1D()(out) 

    out = Dense(output_timesteps)(out) 
    model = Model(inputs=[visible1], outputs=[out])
    
    print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 144, 37)]    0                                            
__________________________________________________________________________________________________
permute (Permute)               (None, 37, 144)      0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 144, 37)      1406        input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 144, 37)      1406        input_1[0][0]                    
______________________________________________________________________________________________

In [15]:
gc.collect()

1503

In [16]:
    model.compile(loss=MAEMD, optimizer='adam', metrics=['mse','mae', MAEMD])
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    batch_size = timesteps
    epochs = 1000
    #root_squared_mean_error
    history = LossHistory()
    history.init()
    
    #hist = model.fit(trX, trY, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history, early_stopping])  # , checkpoint

In [17]:
    %%time
    b_size = timesteps
    #hist = model.fit(strvaX, strvaY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(teX, teY), callbacks=[history, early_stopping])  # , checkpoint
    hist = model.fit(strX, strY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(svaX, svaY), callbacks=[history, early_stopping])  # , checkpoint

Epoch 1/1000
153/153 [==============================] - 82s 411ms/step - loss: 0.2082 - mse: 0.0279 - mae: 0.1180 - MAEMD: 0.2081 - val_loss: 0.1608 - val_mse: 0.0155 - val_mae: 0.0990 - val_MAEMD: 0.1607
Epoch 2/1000
153/153 [==============================] - 94s 617ms/step - loss: 0.1459 - mse: 0.0170 - mae: 0.1036 - MAEMD: 0.1459 - val_loss: 0.1370 - val_mse: 0.0166 - val_mae: 0.1040 - val_MAEMD: 0.1370
Epoch 3/1000
153/153 [==============================] - 97s 631ms/step - loss: 0.1358 - mse: 0.0177 - mae: 0.1070 - MAEMD: 0.1358 - val_loss: 0.1340 - val_mse: 0.0178 - val_mae: 0.1084 - val_MAEMD: 0.1340
Epoch 4/1000
153/153 [==============================] - 96s 630ms/step - loss: 0.1325 - mse: 0.0177 - mae: 0.1071 - MAEMD: 0.1325 - val_loss: 0.1304 - val_mse: 0.0175 - val_mae: 0.1073 - val_MAEMD: 0.1304
Epoch 5/1000
153/153 [==============================] - 97s 636ms/step - loss: 0.1295 - mse: 0.0174 - mae: 0.1060 - MAEMD: 0.1295 - val_loss: 0.1253 - val_mse: 0.0160 - val_mae: 0.

### Saving Basic Model Training Results

In [18]:
model.save('Basic Ramp Model Final_fh12.h5')

In [19]:
gc.collect()

4592

In [20]:
loss_history = hist.history['loss']
valloss_history = hist.history['val_loss']

## Basic Model Test Results

In [21]:
testPredict = model.predict(teX, batch_size=batch_size)

In [22]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [23]:
tePredict = testPredict.reshape(-1)
testY = teY.reshape(-1)
print('Error Test Score > MSE == ', npMSE(testY, tePredict), ' MAE == ', npMAE(testY, tePredict), ' MAEMS == ', npMAEMD(testY, tePredict))

Error Test Score > MSE ==  0.012877223765622883  MAE ==  0.08953137240019018  MAEMS ==  0.09640088260216916


## Wind Ramp Rate FFEL Model

In [24]:
trX, vaX, trY, vaY = train_test_split(trvaX, trvaY, test_size=1/6, shuffle=False)
trY = trY.reshape(-1,output_timesteps)
vaY = vaY.reshape(-1,output_timesteps)

In [25]:
trainPredict = model.predict(trX, batch_size=batch_size)
validPredict = model.predict(vaX, batch_size=batch_size)

In [26]:
e_tr = trainPredict - trY
e_va = validPredict - vaY
errors = np.vstack([e_tr, e_va])
prediction = np.vstack([trainPredict, validPredict])
print(errors.shape)
print(prediction.shape)

(31411, 12)
(31411, 12)


In [27]:
norm_df2 = pd.DataFrame(norm_df).iloc[:prediction.shape[0], :]
norm_df2.columns = ['Normalized Ramp']
norm_df2.head()

,Normalized Ramp
0,0.620197
1,0.544580
2,0.570751
3,0.599594
4,0.495246


In [28]:
Prindex = ['Prediction1', 'Prediction2','Prediction3','Prediction4','Prediction5','Prediction6','Prediction7','Prediction8','Prediction9','Prediction10','Prediction11','Prediction12']
Erindex = ['Error1', 'Error2','Error3','Error4','Error5','Error6','Error7','Error8','Error9','Error10','Error11','Error12']

pr_df = pd.DataFrame(prediction, columns=Prindex)
pr_df

,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6,Prediction7,Prediction8,Prediction9,Prediction10,Prediction11,Prediction12
0,0.576944,0.552743,0.563341,0.493121,0.408325,0.406069,0.521226,0.553002,0.515943,0.567649,0.528050,0.424374
1,0.538577,0.550591,0.492083,0.416955,0.389544,0.506517,0.521203,0.517316,0.591400,0.646456,0.519673,0.419093
2,0.549176,0.526222,0.433467,0.398326,0.593109,0.592674,0.520525,0.508530,0.499884,0.435490,0.386742,0.383212
3,0.538702,0.467881,0.471990,0.658254,0.617754,0.497131,0.489012,0.455205,0.452697,0.421618,0.489446,0.669960
4,0.384145,0.436745,0.623044,0.571493,0.391706,0.415081,0.515201,0.457060,0.404837,0.486248,0.688747,0.758979
...,...,...,...,...,...,...,...,...,...,...,...,...
31406,0.622352,0.684375,0.695968,0.778953,0.746770,0.570206,0.381604,0.400599,0.312627,0.342640,0.400805,0.380967
31407,0.581286,0.647596,0.628323,0.565861,0.510710,0.443483,0.490750,0.420683,0.351688,0.414586,0.442777,0.406025
31408,0.595082,0.636028,0.584006,0.463814,0.386808,0.378333,0.440461,0.414857,0.438971,0.471312,0.378733,0.282537
31409,0.733840,0.687098,0.559241,0.401101,0.386054,0.375634,0.363663,0.421679,0.443951,0.445000,0.467745,0.575575


In [29]:
er_df = pd.DataFrame(errors, columns=Erindex)
er_df

,Error1,Error2,Error3,Error4,Error5,Error6,Error7,Error8,Error9,Error10,Error11,Error12
0,0.025014,-0.059659,-0.049394,-0.102019,-0.104293,-0.003204,-0.066340,-0.160963,-0.083540,0.055031,0.053184,-0.043365
1,-0.073824,-0.062145,-0.103057,-0.095664,-0.019728,-0.081050,-0.192763,-0.082167,0.078781,0.171590,0.051935,0.026303
2,-0.063560,-0.068918,-0.079152,-0.010947,0.005542,-0.121292,-0.078958,-0.004089,0.025018,-0.032248,-0.006049,-0.112257
3,-0.056438,-0.044738,0.062717,0.070687,-0.096211,-0.102352,-0.023607,-0.019661,-0.015041,0.028827,-0.006023,-0.213279
4,-0.128474,0.027473,0.035477,-0.142473,-0.207777,-0.097538,0.040335,-0.010679,0.012047,-0.009221,-0.194492,0.035325
...,...,...,...,...,...,...,...,...,...,...,...,...
31406,-0.036323,0.014351,0.013395,0.174708,0.266570,0.229397,0.088351,-0.034683,-0.395123,-0.243862,-0.044068,-0.120413
31407,-0.088739,-0.034977,0.024077,0.085661,0.169901,0.150230,0.055469,-0.287067,-0.234813,-0.030287,-0.058603,-0.143870
31408,-0.087491,0.031783,0.103806,0.123005,0.093555,-0.056949,-0.267289,-0.171645,-0.005902,-0.030069,-0.171162,-0.154822
31409,0.129595,0.206898,0.218432,0.107848,-0.049227,-0.332115,-0.222838,-0.023194,-0.057429,-0.104896,0.030386,0.088421


In [30]:
norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2.head()

,Normalized Ramp,Prediction1,Prediction2,Prediction3,Prediction4,Prediction5,Prediction6,Prediction7,Prediction8,Prediction9,...,Error3,Error4,Error5,Error6,Error7,Error8,Error9,Error10,Error11,Error12
0,0.620197,0.576944,0.552743,0.563341,0.493121,0.408325,0.406069,0.521226,0.553002,0.515943,...,-0.049394,-0.102019,-0.104293,-0.003204,-0.066340,-0.160963,-0.083540,0.055031,0.053184,-0.043365
1,0.544580,0.538577,0.550591,0.492083,0.416955,0.389544,0.506517,0.521203,0.517316,0.591400,...,-0.103057,-0.095664,-0.019728,-0.081050,-0.192763,-0.082167,0.078781,0.171590,0.051935,0.026303
2,0.570751,0.549176,0.526222,0.433467,0.398326,0.593109,0.592674,0.520525,0.508530,0.499884,...,-0.079152,-0.010947,0.005542,-0.121292,-0.078958,-0.004089,0.025018,-0.032248,-0.006049,-0.112257
3,0.599594,0.538702,0.467881,0.471990,0.658254,0.617754,0.497131,0.489012,0.455205,0.452697,...,0.062717,0.070687,-0.096211,-0.102352,-0.023607,-0.019661,-0.015041,0.028827,-0.006023,-0.213279
4,0.495246,0.384145,0.436745,0.623044,0.571493,0.391706,0.415081,0.515201,0.457060,0.404837,...,0.035477,-0.142473,-0.207777,-0.097538,0.040335,-0.010679,0.012047,-0.009221,-0.194492,0.035325


In [31]:
norm_df22 = pd.DataFrame(norm_df).iloc[prediction.shape[0]+timesteps:, :]
norm_df22.columns = ['Normalized Wind']
npnorm22 = np.array(norm_df22)

In [32]:
pr_df = pd.DataFrame(tePredict.reshape(-1,12))
pr_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.569626,0.486957,0.401701,0.289378,0.291180,0.360890,0.485229,0.414722,0.381292,0.520389,0.389873,0.284724
1,0.444172,0.431101,0.294488,0.176223,0.204284,0.379527,0.398808,0.398596,0.629990,0.477531,0.333508,0.310682
2,0.352566,0.332039,0.286100,0.280012,0.363542,0.344059,0.367165,0.508501,0.404863,0.216263,0.242895,0.373913
3,0.327348,0.363582,0.417275,0.413158,0.329513,0.353855,0.484363,0.345341,0.240361,0.337335,0.498731,0.600383
4,0.315212,0.365783,0.352309,0.271984,0.312949,0.439980,0.351323,0.353594,0.406051,0.468890,0.493729,0.555659
...,...,...,...,...,...,...,...,...,...,...,...,...
3486,0.278861,0.327042,0.339846,0.213460,0.328741,0.390500,0.524273,0.560072,0.535437,0.559962,0.449489,0.410496
3487,0.461214,0.475044,0.405985,0.297391,0.500918,0.541814,0.616275,0.556881,0.532883,0.509687,0.367808,0.312976
3488,0.537132,0.499227,0.442107,0.479796,0.561478,0.438583,0.426091,0.370012,0.211754,0.091271,0.154969,0.443015
3489,0.468449,0.422549,0.447049,0.363853,0.381873,0.310004,0.400983,0.472121,0.379340,0.406832,0.630483,0.676744


In [33]:
teY = testY.reshape(-1,12)
e_te = testPredict-teY
er_df = pd.DataFrame(e_te, columns=Erindex)
er_df

,Error1,Error2,Error3,Error4,Error5,Error6,Error7,Error8,Error9,Error10,Error11,Error12
0,0.228817,0.193704,-0.033581,-0.418372,-0.295321,-0.083983,-0.016151,-0.135174,-0.056068,0.033235,0.036835,-0.045295
1,0.150919,-0.004180,-0.413261,-0.410279,-0.240589,-0.121854,-0.151087,-0.038764,0.142836,0.124493,0.003489,-0.071129
2,-0.082715,-0.375711,-0.300402,-0.164860,-0.137838,-0.205836,-0.070194,0.021347,0.051825,-0.113756,-0.138916,-0.064725
3,-0.380402,-0.222919,-0.027598,-0.088223,-0.220383,-0.083505,-0.002791,-0.007697,-0.089658,-0.044476,0.060093,0.056882
4,-0.271290,-0.079090,-0.149071,-0.277911,-0.124411,-0.047174,-0.001715,0.023575,0.024240,0.030251,-0.049773,0.012557
...,...,...,...,...,...,...,...,...,...,...,...,...
3486,-0.128636,-0.034543,-0.015736,-0.210719,0.113702,-0.066886,-0.070851,0.086396,0.059967,0.019950,-0.103541,-0.170675
3487,0.099630,0.119461,-0.018193,0.082353,0.043533,-0.053309,0.142598,0.081411,-0.007129,-0.043343,-0.213363,-0.267882
3488,0.181549,0.075048,0.227069,0.022411,-0.033645,-0.035094,-0.049379,-0.170001,-0.341276,-0.489900,-0.425889,-0.331628
3489,0.044270,0.207511,-0.010336,-0.231270,-0.091804,-0.165466,-0.139029,-0.080910,-0.201830,-0.174026,-0.144160,-0.058768


In [34]:
prnorm = np.array(pr_df)
ernorm =np.array(er_df)
print(ernorm.shape)

(3491, 12)


In [35]:
etedat = np.concatenate((npnorm22[:prnorm.shape[0],:], prnorm, ernorm), axis=1)
#etedat[169:171,:]

In [36]:
eteX, eteY = create_dataset(etedat, timesteps, 1, 0)
eteY.shape

(3345, 1, 25)

In [37]:
eteY = eteY[:,:,-12:].reshape(-1,12)

In [38]:
norm_df2 = np.array(norm_df2)
output_timesteps = 1
Xe, Ye = create_dataset(norm_df2, timesteps, 1, 0)
Ye.shape

(31265, 1, 25)

In [39]:
Ye = Ye[:,:,-12:].reshape(-1, 12)
print(Xe.shape)
print(Ye.shape)

(31265, 144, 25)
(31265, 12)


In [40]:
trXe, vaXe, trYe, vaYe = train_test_split(Xe, Ye, test_size=0.3, shuffle=False)

In [41]:
gc.collect()

6278

In [42]:
eteX.shape, eteY.shape, Xe.shape, Ye.shape

((3345, 144, 25), (3345, 12), (31265, 144, 25), (31265, 12))

In [43]:
def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))*10+K.mean(K.abs(y_pred - y_true))

In [44]:
num_features = Xe.shape[2]

with tf.device('/gpu:0'):
    alpha=1.0
    gamma=1.2
    beta = 1
    hfilters = 256
    hkernel_size1 = 1
    hkernel_size2 = 2
    visible1e = Input(shape=(timesteps, num_features))
   
    per1e = Permute((2,1))(visible1e)
    den1ae = Dense(timesteps, activation='tanh')(per1e)
    den1be = Dense(timesteps, activation='sigmoid')(per1e)
    den1e = Multiply()([den1ae, den1be])
    per2e = Permute((2,1), name='attention_vec')(den1e)
    mul1e = Multiply()([visible1e, per2e])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=1)(mul1e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    res01ae = Add()([visible1e, d1e])   # (100, 25) (100, 25)
    res01be = Subtract()([visible1e, d1e])

    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01ae)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    
    d1e = Multiply()([d1ae, d1be])
    res02ae = Add()([res01ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res01be) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    res02be = Subtract()([res01be, d2e])   # (100, 25) (100, 25) 
    res02e = Concatenate()([res02ae, res02be, res01ae, res01be])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=8)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    res03ae = Add()([res02ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res02e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=8)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    res03be = Subtract()([res02be, d2e])   # (100, 25) (100, 25)
    res03e = Concatenate()([res03ae, res03be, res02e])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=8)(res03e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    res04ae = Add()([res03ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=8)(res03e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=4)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    res04be = Subtract()([res02be, d2e])   # (100, 25) (100, 25)
    res04e = Concatenate()([res04ae, res04be, res03e])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res04e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    res05ae = Add()([res04ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=4)(res04e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=2)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    
    res05be = Subtract()([res04be, d2e])   # (100, 25) (100, 25)
    res05e = Concatenate()([res05ae, res05be, res04e])
    
    d1e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res05e)
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])
    
    d1e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=1)(d1e)    
    d1ae = Activation(activations.tanh)(d1e)
    d1be = Activation(activations.sigmoid)(d1e)
    d1e = Multiply()([d1ae, d1be])

    res06ae = Add()([res05ae, d1e])   # (100, 25) (100, 25)
    
    d2e = Conv1D(filters=round(hfilters*alpha), kernel_size=round(hkernel_size1*beta), padding='causal', dilation_rate=2)(res05e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])
    d2e = Conv1D(filters=num_features, kernel_size=round(hkernel_size2*beta), padding='causal', dilation_rate=1)(d2e) 
    d2ae = Activation(activations.tanh)(d2e)
    d2be = Activation(activations.sigmoid)(d2e)
    d2e = Multiply()([d2ae, d2be])

    res06be = Subtract()([res05be, d2e])   # (100, 25) (100, 25)
    res06e = Concatenate()([res05ae, res05be])
    
    res10e = Concatenate()([res02e, res03e, res04e, res05e, res06e])   # 
    
    #print('res10 :', res10.shape)  # (None, 24, 11) 
    
    oute = Conv1D(timesteps*5, 1, padding='same', activation=PReLU())(res10e)   # 256, 11X10=110
    oute = Dropout(0.2)(oute)   #SpatialDropout1D
    
    oute = Conv1D(timesteps*3, 1, padding='same', activation=PReLU())(oute) # 512,  110X5=550
    oute = Dropout(0.2)(oute)
    
    oute = GlobalAveragePooling1D()(oute) # pool_size=2, strides=1
    
    oute = Dense(12)(oute) 
    modele = Model(inputs=[visible1e], outputs=[oute])
    
    print(modele.summary())
    
    modele.compile(loss=mse_mae, optimizer='adam', metrics=['mse','mae','mape'])
    early_stopping =EarlyStopping(monitor='val_loss', patience=10)
    batch_size = 168
    epochs = 1000

    history_e = LossHistory()
    history_e.init()
    
    #hist = model.fit(trX, trY, epochs=epochs, batch_size=batch_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history, early_stopping])  # , checkpoint

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 144, 25)]    0                                            
__________________________________________________________________________________________________
permute_2 (Permute)             (None, 25, 144)      0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 25, 144)      20880       permute_2[0][0]                  
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 25, 144)      20880       permute_2[0][0]                  
____________________________________________________________________________________________

In [45]:
gc.collect()

2538

In [46]:
# %%time
    histe = modele.fit(trXe, trYe, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(vaXe, vaYe), callbacks=[history_e, early_stopping])  # , checkpoint

Epoch 1/1000
152/152 [==============================] - 266s 2s/step - loss: 0.2483 - mse: 0.0159 - mae: 0.0898 - mape: 419.2016 - val_loss: 0.1575 - val_mse: 0.0085 - val_mae: 0.0726 - val_mape: 951.5339
Epoch 2/1000
152/152 [==============================] - 276s 2s/step - loss: 0.1628 - mse: 0.0089 - mae: 0.0738 - mape: 423.5485 - val_loss: 0.1264 - val_mse: 0.0064 - val_mae: 0.0626 - val_mape: 1053.7777
Epoch 3/1000
152/152 [==============================] - 289s 2s/step - loss: 0.1370 - mse: 0.0071 - mae: 0.0657 - mape: 470.0483 - val_loss: 0.1076 - val_mse: 0.0051 - val_mae: 0.0561 - val_mape: 979.6042
Epoch 4/1000
152/152 [==============================] - 275s 2s/step - loss: 0.1233 - mse: 0.0062 - mae: 0.0613 - mape: 441.9853 - val_loss: 0.0975 - val_mse: 0.0045 - val_mae: 0.0524 - val_mape: 1178.0166
Epoch 5/1000
152/152 [==============================] - 274s 2s/step - loss: 0.1150 - mse: 0.0056 - mae: 0.0585 - mape: 433.5725 - val_loss: 0.0913 - val_mse: 0.0041 - val_mae: 0

### Saving FFEL Model Training Results

In [47]:
eloss_history = histe.history['loss']
valeloss_history = histe.history['val_loss']

In [48]:
modele.save('Error Learning Ramp Model_fh12.h5')

In [49]:
gc.collect()

4570

## FFEL Model Test Results

In [50]:
trainePredict = modele.predict(Xe, batch_size=batch_size)
etePredict = modele.predict(eteX, batch_size=batch_size)

In [51]:
trePredict = trainePredict.reshape([-1])
trainYe = Ye.reshape([-1])

print('Error Training Score > MSE == ', (np.mean(np.square(trainYe-trePredict))), ' MAE == ', mean_absolute_error(trainYe,trePredict), ' RMSE == ', np.sqrt(np.mean(np.square(trainYe-trePredict))))

Error Training Score > MSE ==  0.003695841136758344  MAE ==  0.0473448617423577  RMSE ==  0.06079343004600369


In [52]:
etestPredict = etePredict.reshape([-1])
testYe = eteY.reshape([-1])

print('Error Training Score > MSE == ', (np.mean(np.square(testYe-etestPredict))), ' MAE == ', mean_absolute_error(testYe,etestPredict), ' RMSE == ', np.sqrt(np.mean(np.square(testYe-etestPredict))))

Error Training Score > MSE ==  0.004368571359662568  MAE ==  0.05136803931860675  RMSE ==  0.06609516895857494


## Final Prediction Results

In [53]:
testPredict = tePredict.reshape(-1,12)
addtestPredict = -etePredict + testPredict[timesteps:-2,:]
print(addtestPredict.shape)

(3345, 12)


In [54]:
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-testPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), 'MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]))
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-addtestPredict))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], addtestPredict), 'MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], addtestPredict))

Error Test Score > MSE ==  0.012836964015489147  MAE ==  0.08934994887150143 MAPE ==  18.053389355078796
Error Test Score > MSE ==  0.0043685713599015  MAE ==  0.051368039296910056 MAPE ==  10.427107203598556
